In [6]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

In [7]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [8]:
analysis_date = dt.date(2022, 1, 12)

In [9]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [10]:
not_ran = [208, 343, 273, 372, 346, 380, 243, 45, 260, 75]

In [11]:
for agency in not_ran:
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date)
        rt_day.set_filter(start_time='15:00', end_time='19:00', direction_id='1')
        rt_day.filter_formatted = ', PM Peak, Jan 12 (Wed)'
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_01_12_pm_peak.html')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        print(e)

calculating for agency: 208...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 208
vehicle positions trip ids not in schedule
calculating for agency: 343...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 343
vehicle positions trip ids not in schedule
calculating for agency: 273...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 273
vehicle positions trip ids not in schedule
calculating for agency: 372...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 372
vehicle positions trip ids not in schedule
calculating for agency: 346...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 346
vehicle positions trip ids not in schedule
calculating for agency: 380...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 380
vehicle positions trip ids not in schedule
calculating for agency: 243...
found parquet
found parquet
found parqu

/home/jovyan/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip BCT109 NB_MF.T17
cannot convert float infinity to integer


/home/jovyan/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip BCT109 NB_MF.T07
cannot convert float infinity to integer
could not generate delays for trip BCT102 SB_MF.T04
Columns must be same length as key
view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': '1', 'direction': None}
rt failed for agency 260
'DataFrame' object has no attribute 'set_crs'
calculating for agency: 75...
found parquet
found parquet
found parquet
found parquet
found_parquet
projecting
done
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 100-1
check location data for trip 200-1
check location data for trip 200-1
check location data for trip 200-1
check location data for trip 200-1
check location data for trip 200-1
check location data for trip 200-1
